<a href="https://colab.research.google.com/github/aasiyasan/Training/blob/main/Audiofiles_comparison_using_torchaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchaudio faiss-cpu transformers pillow tqdm

In [ ]:
!apt install libomp-dev
!pip install faiss-cpu

In [ ]:
import os
import requests
from io import BytesIO
from tqdm import tqdm

import torch
from torchaudio import transforms
from transformers import CLIPProcessor, CLIPModel
import faiss
import numpy as np

In [ ]:
# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Sample dataset of audio URLs
audio_urls = [
    "https://freeanimalsounds.org/download/95/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/115/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/124/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/150/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/165/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/218/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/220/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/227/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/223/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/230/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/258/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/270/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/271/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/272/?tmstv=1757038065",
    "https://freeanimalsounds.org/download/250/?tmstv=1757038065"
]


In [ ]:
os.makedirs("dataset", exist_ok=True)
audio_paths = []
audio_names = []
# Download audio files
for i, url in enumerate(tqdm(audio_urls, desc="Downloading animal sounds")):
    response = requests.get(url)
    if response.status_code == 200:
        path = f"dataset/audi_{i}.mp3"
        with open(path, "wb") as f:
            f.write(response.content)
        audio_paths.append(path)

for p in audio_paths:
    audio_names.append(p)

In [ ]:

from transformers import Wav2Vec2Model, Wav2Vec2Processor

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "facebook/wav2vec2-base-960h"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name).to(device)


In [ ]:
import librosa
import numpy as np

def get_audio_embedding(path):
    waveform, sr = librosa.load(path, sr=16000)
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state  # shape: (batch, time, features)
        embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()  # mean pooling
    return embedding


In [ ]:
import faiss

embeddings = []

for path in tqdm(audio_paths, desc="Embedding audios"):
    vec = get_audio_embedding(path)
    embeddings.append(vec)

embeddings = np.stack(embeddings).astype("float32")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, "audio_index.faiss")


In [ ]:
import torchaudio
def search_similar_audio(query_path, top_k=3):
    index = faiss.read_index("audio_index.faiss")

    query_vec = get_audio_embedding(query_path).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vec, top_k)

    print(f"Query audio: {query_path}")
    print("Top matches:")
    for idx, dist in zip(indices[0], distances[0]):
        print(f"{audio_names[idx]} - Distance: {dist}")
        #torchaudio.io.play_audio(torchaudio.load(audio_urls[idx]))


In [ ]:
search_similar_audio("/content/horse-neigh.mp3")

Query audio: /content/horse-neigh.mp3
Top matches:
dataset/audi_14.mp3 - Distance: 1.3530620336532593
dataset/audi_11.mp3 - Distance: 2.0406339168548584
dataset/audi_10.mp3 - Distance: 2.6627511978149414
